In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    
from tensorflow.keras import datasets
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import *


In [7]:
model_name = 'inception'

cifar100_model_train = np.load(f'./data/CIFAR100_{model_name}_train_feat.npy')
cifar100_model_test = np.load(f'./data/CIFAR100_{model_name}_test_feat.npy')
print(cifar100_model_train.shape, cifar100_model_test.shape)

cifar100_vanilla_train = np.load(f'./data/CIFAR100_vae_vanilla_{model_name}_encoding_train.npy')
cifar100_vanilla_test = np.load(f'./data/CIFAR100_vae_vanilla_{model_name}_encoding_test.npy')
print(cifar100_vanilla_train.shape, cifar100_vanilla_test.shape)

cifar100_injected_train = np.load(f'./data/CIFAR100_vae_injected_{model_name}_encoding_train.npy')
cifar100_injected_test = np.load(f'./data/CIFAR100_vae_injected_{model_name}_encoding_test.npy')
print(cifar100_injected_train.shape, cifar100_injected_test.shape)

cifar100_adapted_train = np.load(f'./data/CIFAR100_vae_adapted_{model_name}_encoding_train.npy')
cifar100_adapted_test = np.load(f'./data/CIFAR100_vae_adapted_{model_name}_encoding_test.npy')
print(cifar100_adapted_train.shape, cifar100_adapted_test.shape)

(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()


(50000, 2048) (10000, 2048)
(50000, 2048) (10000, 2048)
(50000, 2048) (10000, 2048)
(50000, 2048) (10000, 2048)


In [11]:
input_shape = (cifar100_model_train.shape[1],)
batch_size = 128
epochs = 100
val_split=0.2
lr = 1e-3
callbacks = [
    #ReduceLROnPlateau(patience=5, factor=0.1),
   # EarlyStopping(patience=10)
]



In [ ]:
%%time

model = Sequential( [
    Input(shape=input_shape),
    Dense(128, activation='relu'),
    BatchNormalization(),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    
    Dense(100, activation='softmax')
], name=model_name)

model.compile(
    optimizer=Adam(lr=lr),  
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

#model.summary()

model_hist = model.fit(
    x=cifar100_model_train, 
    y=train_labels, 
    batch_size=batch_size, 
    epochs=epochs,
    validation_split=val_split,
    callbacks=callbacks
)

model_eval = model.evaluate(
    x=cifar100_model_test,
    y=test_labels,
    batch_size=batch_size
)

print(f'{model.name} test - loss: {model_eval[0]}, accuracy: {model_eval[1]}')

In [ ]:
%%time

vanilla_model = Sequential([
    Input(shape=input_shape),
    Dense(128, activation='relu'),
    BatchNormalization(),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(100, activation='softmax')
], name='vanilla')

vanilla_model.compile(
    optimizer=Adam(lr=lr),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

#vanilla_model.summary()

vanilla_hist = vanilla_model.fit(
    x=cifar100_vanilla_train, 
    y=train_labels, 
    batch_size=batch_size, 
    epochs=epochs,
    validation_split=val_split,
    callbacks=callbacks
)

vanilla_model_eval = vanilla_model.evaluate(
    x=cifar100_vanilla_test, 
    y=test_labels,
    batch_size=batch_size
)

print(f'{vanilla_model.name} test - loss: {vanilla_model_eval[0]}, accuracy: {vanilla_model_eval[1]}')

In [19]:
%%time

injected_model = tf.keras.Sequential([
    Input(shape=input_shape),
    Dense(16, activation='relu'),
    BatchNormalization(),
    
    Dense(16, activation='relu'),
    BatchNormalization(),

    Dense(100, activation='softmax')
], name='injected')

injected_model.compile(
    optimizer=Adam(lr=lr),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

#injected_model.summary()

injected_hist = injected_model.fit(
    x=cifar100_injected_train, 
    y=train_labels, 
    batch_size=batch_size, 
    epochs=epochs,
    validation_split=val_split,
    callbacks=callbacks
)

injected_model_eval = injected_model.evaluate(
    x=cifar100_injected_test, 
    y=test_labels,
    batch_size=batch_size
)

print(f'{injected_model.name} test - loss: {injected_model_eval[0]}, accuracy: {injected_model_eval[1]}')

Epoch 1/100
313/313 [==============================] - 2s 5ms/step - loss: 4.6733 - accuracy: 0.0125 - val_loss: 4.5637 - val_accuracy: 0.0219
Epoch 2/100
313/313 [==============================] - 1s 5ms/step - loss: 4.4390 - accuracy: 0.0358 - val_loss: 4.5011 - val_accuracy: 0.0300
Epoch 3/100
313/313 [==============================] - 1s 5ms/step - loss: 4.3040 - accuracy: 0.0503 - val_loss: 4.4872 - val_accuracy: 0.0342
Epoch 4/100
313/313 [==============================] - 1s 5ms/step - loss: 4.1873 - accuracy: 0.0634 - val_loss: 4.5074 - val_accuracy: 0.0325
Epoch 5/100
313/313 [==============================] - 1s 5ms/step - loss: 4.0963 - accuracy: 0.0746 - val_loss: 4.5420 - val_accuracy: 0.0347
Epoch 6/100
313/313 [==============================] - 1s 5ms/step - loss: 4.0272 - accuracy: 0.0834 - val_loss: 4.5853 - val_accuracy: 0.0334
Epoch 7/100
313/313 [==============================] - 1s 5ms/step - loss: 3.9601 - accuracy: 0.0906 - val_loss: 4.6283 - val_accuracy: 0.0331

KeyboardInterrupt: 

In [20]:
%%time

adapted_model = Sequential([
    Input(shape=input_shape),
    Dense(16, activation='relu'),
    BatchNormalization(),
    
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(100, activation='softmax')
],name='adapted' )

adapted_model.compile(
    optimizer=Adam(lr=lr),             
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

#adapted_model.summary()

adapted_hist = adapted_model.fit(
    x=cifar100_adapted_train, 
    y=train_labels, 
    batch_size=batch_size, 
    epochs=epochs,
    validation_split=val_split,
    callbacks=callbacks
)

adapted_model_eval = adapted_model.evaluate(
    x=cifar100_adapted_test, 
    y=test_labels,
    batch_size=batch_size
)

print(f'{adapted_model.name} test - loss: {adapted_model_eval[0]}, accuracy: {adapted_model_eval[1]}')

Epoch 1/100
313/313 [==============================] - 2s 5ms/step - loss: 4.6755 - accuracy: 0.0134 - val_loss: 4.5253 - val_accuracy: 0.0245
Epoch 2/100
313/313 [==============================] - 1s 5ms/step - loss: 4.4146 - accuracy: 0.0388 - val_loss: 4.4326 - val_accuracy: 0.0341
Epoch 3/100
313/313 [==============================] - 1s 5ms/step - loss: 4.2510 - accuracy: 0.0606 - val_loss: 4.3804 - val_accuracy: 0.0423
Epoch 4/100
313/313 [==============================] - 1s 5ms/step - loss: 4.0947 - accuracy: 0.0815 - val_loss: 4.3488 - val_accuracy: 0.0477
Epoch 5/100
313/313 [==============================] - 1s 5ms/step - loss: 3.9774 - accuracy: 0.0918 - val_loss: 4.3661 - val_accuracy: 0.0494
Epoch 6/100
313/313 [==============================] - 1s 5ms/step - loss: 3.8898 - accuracy: 0.1056 - val_loss: 4.3838 - val_accuracy: 0.0505
Epoch 7/100
313/313 [==============================] - 1s 5ms/step - loss: 3.8269 - accuracy: 0.1128 - val_loss: 4.4215 - val_accuracy: 0.0545

Epoch 58/100
313/313 [==============================] - 2s 5ms/step - loss: 3.2841 - accuracy: 0.1841 - val_loss: 5.3275 - val_accuracy: 0.0499
Epoch 59/100
313/313 [==============================] - 2s 5ms/step - loss: 3.2650 - accuracy: 0.1873 - val_loss: 5.3268 - val_accuracy: 0.0478
Epoch 60/100
313/313 [==============================] - 1s 5ms/step - loss: 3.2661 - accuracy: 0.1870 - val_loss: 5.3142 - val_accuracy: 0.0485
Epoch 61/100
313/313 [==============================] - 1s 5ms/step - loss: 3.2743 - accuracy: 0.1841 - val_loss: 5.3164 - val_accuracy: 0.0480
Epoch 62/100
313/313 [==============================] - 1s 5ms/step - loss: 3.2630 - accuracy: 0.1882 - val_loss: 5.3557 - val_accuracy: 0.0481
Epoch 63/100
313/313 [==============================] - 1s 5ms/step - loss: 3.2663 - accuracy: 0.1859 - val_loss: 5.3704 - val_accuracy: 0.0472
Epoch 64/100
313/313 [==============================] - 1s 5ms/step - loss: 3.2521 - accuracy: 0.1890 - val_loss: 5.3541 - val_accuracy:

In [ ]:

metric = 'accuracy'
plt.plot(model_hist.history['loss'], label=f'InceptionNet')
plt.plot(vanilla_hist.history['loss'], label='BiModal')
plt.plot(injected_hist.history['loss'], label='Injected')
plt.plot(adapted_hist.history['loss'], label='Adapted')
plt.legend()
plt.title(f'{model_name } training Loss')
plt.savefig(f'./plots/{model_name}/loss.png')
plt.show()

plt.plot(model_hist.history[f'{metric}'], label=f'InceptionNet')
plt.plot(vanilla_hist.history[f'{metric}'], label='BiModal')
plt.plot(injected_hist.history[f'{metric}'], label='Injected')
plt.plot(adapted_hist.history[f'{metric}'], label='Adpated')
plt.legend()
plt.title(f'{model_name} training accuracy')
plt.savefig(f'./plots/{model_name}/accuracy.png')
plt.show()



In [ ]:
test_data = {
    'model': [f'{model_name}', f'{model_name}_vanilla', f'{model_name}_injected', f'{model_name}_adapted'],
    f'loss': [model_eval[0], vanilla_model_eval[0], injected_model_eval[0], adapted_model_eval[0]],
    f'acc': [model_eval[1], vanilla_model_eval[1], injected_model_eval[1], adapted_model_eval[1]]
}

test_df = pd.DataFrame(data=test_data)
                          
test_df.to_csv(f'results/{model_name}_test_df.csv')        

test_df.head()

In [ ]:
full_df = pd.DataFrame(columns=['model', 'loss', 'acc'])

for m in ['mobilenet', 'resnet', 'efficientnet', 'inception']:
    df = pd.read_csv(f'results/{m}_test_df.csv')
    full_df = pd.concat([full_df, df])

full_df.drop('Unnamed: 0', axis=1, inplace=True)
full_df.reset_index(inplace=True, drop=True)

full_df.to_csv(f'results/full_df.csv') 

full_df.head(20)